In [2]:
!pip install folium


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [3]:
# Get key from geoapify.com into ./geoapify.key
KEY_PATH = "./geoapify.key"
API_KEY = open(KEY_PATH).read().strip()

In [4]:
import requests
from requests.structures import CaseInsensitiveDict

def get_geojson(mode, lat, lon, minutes):
    seconds = minutes * 60
    url = "https://api.geoapify.com/v1/isoline?lat=%f&lon=%f&type=time&mode=%s&range=%d&apiKey=%s" % (lat,lon,mode,seconds, API_KEY)

    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"

    resp = requests.get(url, headers=headers)
    return json.loads(resp.content)["features"][0]["geometry"]


In [5]:
# Execute the query
response = requests.get("http://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A25%5D%3B%0Aarea%28id%3A3600092277%29-%3E.searchArea%3B%0A%28%0A%20%20node%5B%22railway%22%3D%22station%22%5D%28area.searchArea%29%3B%0A%20%20way%5B%22railway%22%3D%22station%22%5D%28area.searchArea%29%3B%0A%20%20relation%5B%22railway%22%3D%22station%22%5D%28area.searchArea%29%3B%0A%29%3B%0Aout%20body%3B%0A%3E%3B%0Aout%20skel%20qt%3B")

# Check the status code of the response
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()
    
    # Extract the nodes from the data
    nodes = data['elements']
    
    # Create a list to store the train stations
    train_stations = []
    
    # Iterate through the nodes
    for node in nodes:
        try:
            # Extract the coordinates and name of the train station
            lat = node['lat']
            lon = node['lon']
            name = node['tags'].get('name', '')

            # Add the train station to the list
            train_stations.append((lat, lon, name))
        except KeyError:
            pass

else:
    # Print an error message
    print("Error:", response.status_code)
    print(response.text)

In [ ]:
import json

bicycle_polygons = []
for t in train_stations:
    five_minutes = get_geojson("bicycle", t[0],t[1], 5)
    ten_minutes = get_geojson("bicycle", t[0],t[1], 10)
    fifteen_minutes = get_geojson("bicycle", t[0],t[1], 15)
    bicycle_polygons.append((t[0],t[1],five_minutes,ten_minutes,fifteen_minutes,t[2]))

walk_polygons = []
for t in train_stations:
    five_minutes = get_geojson("walk", t[0],t[1], 5)
    ten_minutes = get_geojson("walk", t[0],t[1], 10)
    fifteen_minutes = get_geojson("walk", t[0],t[1], 15)
    walk_polygons.append((t[0],t[1],five_minutes,ten_minutes,fifteen_minutes,t[2]))


In [ ]:
import folium
import io
from PIL import Image

def generate_png(mode, polygon):
    
    mp = polygon

    lat = mp[0]
    lon = mp[1]

    five_minutes = mp[2]
    ten_minutes = mp[3]
    fifteen_minutes = mp[4]
    name = mp[5]

    if mode == "walk":
        zoom_level = 15
    else:
        zoom_level = 13
    
    loc = mode.capitalize() + ': ' + name
    title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc) 


    m = folium.Map(location=[lat,lon], zoom_start=zoom_level)


    folium.TileLayer('cartodbpositron').add_to(m)

    folium.GeoJson(five_minutes, style_function=lambda feature: {
            'fillColor': '#2735B5',
            'color': '#2735B5',
            'weight': 2,
        }).add_to(m)
    folium.GeoJson(ten_minutes, style_function=lambda feature: {
            'fillColor': '#5C7CCF',
            'color': '#5C7CCF',
            'weight': 2,
        }).add_to(m)
    folium.GeoJson(fifteen_minutes, style_function=lambda feature: {
            'fillColor': '#81A7E0',
            'color': '#81A7E0',
            'weight': 2,
        }).add_to(m)
    folium.Marker( location=[lat,lon],
                 popup=folium.Popup(name, max_width=2650)
                 ).add_to(m)

    footer_html = "Made by OpenTech Community Thailand CC-BY-SA <BR> Map(c) OpenStreetMap Contributors ODbL"
    m.get_root().html.add_child(folium.Element(title_html))
    m.get_root().html.add_child(folium.Element(footer_html))

    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    
    # Make sure you've created output/bicycle and output/walk
    output_filename = "output" + '/' + mode + '/' + name + ".png"
    
    img.save(output_filename)

    print(output_filename)


In [ ]:
for m in bicycle_polygons:
    generate_png("bicycle",m)

In [ ]:
for m in walk_polygons:
    generate_png("walk",m)